Here we are writing the naive algorithm to generate all possible moves for each gamestate and check if the move is valid. The move is valid if after it is executed, the king is not in check. Note that this algorithm is extremely inefficient, as it requires regenerating all possible moves two times (very expensive function!).

Firstly, we define the GameState class to describe the current gamestate with initial variables.

In [ ]:
class GameState():
    def __init__(self):
        self.board = [
            ["bR","bN","bB","bQ","bK","bB","bN","bR"],
            ["bp","bp","bp","bp","bp","bp","bp","bp"],
            ["--","--","--","--","--","--","--","--"],
            ["--","--","--","--","--","--","--","--"],
            ["--","--","--","--","--","--","--","--"],
            ["--","--","--","--","--","--","--","--"],
            ["wp","wp","wp","wp","wp","wp","wp","wp"],
            ["wR","wN","wB","wQ","wK","wB","wN","wR"],
        ]
        self.moveFunctions = {'p':self.getPawnMoves,'R':self.getRookMoves,'N':self.getKnightMoves,'B':self.getBishopMoves,'Q':self.getQueenMoves,'K':self.getKingMoves}
        self.whiteToMove = True
        self.moveLog = []
        self.whiteKingLocation = (7,4)
        self.blackKingLocation = (0,4)
        self.checkmate = False
        self.stalemate = False

We then create the class Move to store the information of each move. The information includes the initial and final squares of the piece, its moveID and conversion of the information to conventional notation (for example, the top left square of the chessboard is a8). The moveID is used to compare two Move object if they are equal to each other.

In [ ]:
class Move():
    ranksToRows = {"1":7,"2":6,"3":5,"4":4,"5":3,"6":2,"7":1,"8":0}
    rowsToRanks = {v: k for k, v in ranksToRows.items()}
    filesToCols = {"a":0,"b":1,"c":2,"d":3,"e":4,"f":5,"g":6,"h":7}
    colsToFiles = {v: k for k, v in filesToCols.items()}

    def __init__(self, startSq, endSq, board):
        self.startRow = startSq[0]
        self.startCol = startSq[1]
        self.endRow = endSq[0]
        self.endCol = endSq[1]
        self.pieceMoved = board[self.startRow][self.startCol]
        self.pieceCaptured = board[self.endRow][self.endCol]
        self.moveID = self.startRow * 1000 + self.startCol * 100 + self.endRow * 10 + self.endCol

    def __eq__(self, other):
        if isinstance(other, Move):
            return self.moveID == other.moveID
        else:
            return False

    def getChessNotation(self):
        return self.getRankFile(self.startRow, self.startCol) + self.getRankFile(self.endRow,self.endCol)
    
    def getRankFile(self, r, c):
        return self.colsToFiles[c]+self.rowsToRanks[r]

Here, by the laws of chess, we define the move function for each piece.

In [ ]:
Class GameState(GameState):
    def getAllPossibleMoves(self):
        moves=[]
        for r in range(8):
            for c in range(8):
                turn = self.board[r][c][0]
                if (turn =='w' and self.whiteToMove) or (turn=='b' and not self.whiteToMove):
                    piece = self.board[r][c][1]
                    self.moveFunctions[piece](r,c,moves)
        return moves

    def getPawnMoves(self,r,c,moves):
        if self.whiteToMove:
            if self.board[r-1][c]=="--":            #empty square in front of white pawn
                moves.append(Move((r,c),(r-1,c),self.board))
                if self.board[r-2][c]=="--":  
                    moves.append(Move((r,c),(r-2,c),self.board))
            #capture for white                                        
            if c-1>=0:
                if self.board[r-1][c-1][0]=='b':
                    moves.append(Move((r,c),(r-1,c-1),self.board))
            if c+1<=7:
                if self.board[r-1][c+1][0]=='b':
                    moves.append(Move((r,c),(r-1,c+1),self.board))
        else:
            if self.board[r+1][c]=="--":            #empty square in front of black pawn
                moves.append(Move((r,c),(r+1,c),self.board))
                if self.board[r+2][c]=="--":  
                    moves.append(Move((r,c),(r+2,c),self.board))
            #capture for black                                       
            if c-1>=0:
                if self.board[r-1][c-1][0]=='b':
                    moves.append(Move((r,c),(r+1,c-1),self.board))
            if c+1<=7:
                if self.board[r-1][c+1][0]=='b':
                    moves.append(Move((r,c),(r+1,c+1),self.board))

    def getRookMoves(self,r,c,moves):
        directions = ((-1,0),(1,0),(0,-1),(0,1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1,8):
                endRow = r + d[0]*i
                endCol = c + d[1]*i
                if 0<=endRow<8 and 0<=endCol<8:
                    if self.board[endRow][endCol]=='--':
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                    elif self.board[endRow][endCol][0]==enemyColor:
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                        break
                    else:
                        break
                else:
                    break
        
    def getKnightMoves(self,r,c,moves):
        directions = ((-1,2),(-1,-2),(1,-2),(1,2),(-2,1),(-2,-1),(2,-1),(2,1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            endRow = r + d[0]
            endCol = c + d[1]
            if 0<=endRow<8 and 0<=endCol<8:
                if self.board[endRow][endCol]=='--' or self.board[endRow][endCol][0]==enemyColor:
                    moves.append(Move((r,c),(endRow,endCol),self.board))

    def getBishopMoves(self,r,c,moves):
        directions = ((-1,1),(-1,-1),(1,-1),(1,1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1,8):
                endRow = r + d[0]*i
                endCol = c + d[1]*i
                if 0<=endRow<8 and 0<=endCol<8:
                    if self.board[endRow][endCol]=='--':
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                    elif self.board[endRow][endCol][0]==enemyColor:
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                        break
                    else:
                        break
                else:
                    break

    def getQueenMoves(self,r,c,moves):
        directions = ((-1,1),(-1,-1),(1,-1),(1,1),(-1,0),(1,0),(0,1),(0,-1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1,8):
                endRow = r + d[0]*i
                endCol = c + d[1]*i
                if 0<=endRow<8 and 0<=endCol<8:
                    if self.board[endRow][endCol]=='--':
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                    elif self.board[endRow][endCol][0]==enemyColor:
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                        break
                    else:
                        break
                else:
                    break

    def getKingMoves(self,r,c,moves):
        directions = ((-1,1),(-1,-1),(1,-1),(1,1),(1,0),(0,1),(-1,0),(-1,-1))
        enemyColor = "b" if self.whiteToMove else "w"
        for d in directions:
            endRow = r + d[0]
            endCol = c + d[1]
            if 0<=endRow<8 and 0<=endCol<8:
                if self.board[endRow][endCol]=='--' or self.board[endRow][endCol][0]==enemyColor:
                    moves.append(Move((r,c),(endRow,endCol),self.board))

The key to this algorithm is to generate all possible moves, then for each move we "hypothetically" play it. If it puts our king in check, it is invalid and be removed from the list of all possible moves. After that, we undo the move so the gamestate doesn't change after the "hypothetical" move. 

In [ ]:
Class GameState(GameState):
    def inCheck(self):
        if self.whiteToMove:
            return self.squareUnderAttack(self.whiteKingLocation[0], self.whiteKingLocation[1])
        else:
            return self.squareUnderAttack(self.blackKingLocation[0], self.blackKingLocation[1])

        #determine if the enemy can attack the square
    def squareUnderAttack(self, r, c):
        self.whiteToMove = not self.whiteToMove
        oppMoves = self.getAllPossibleMoves()
        for move in oppMoves:
            if move.endRow == r and move.endCol == c:
                self.whiteToMove = not self.whiteToMove
                return True
        self.whiteToMove = not self.whiteToMove
        return False

    def makeMove(self, move):
        self.board[move.startRow][move.startCol]= "--"
        self.board[move.endRow][move.endCol] = move.pieceMoved
        self.moveLog.append(move)
        self.whiteToMove = not self.whiteToMove 

        #update the king's location
        if move.pieceMoved=='wK':
            self.whiteKingLocation=(move.endRow,move.endCol)
        elif move.pieceMoved=='bK':
            self.blackKingLocation=(move.endRow,move.endCol)

    #undo the last move made
    def undoMove(self):
        if len(self.moveLog)!=0:
            move = self.moveLog.pop()
            self.board[move.startRow][move.startCol]= move.pieceMoved
            self.board[move.endRow][move.endCol] = move.pieceCaptured
            self.whiteToMove = not self.whiteToMove 
            #track white and black king postition!

Finally we have the getValidMoves function as below. It takes the list from getAllPossibleMoves, check for validity and return the list after removing invalid moves. The remove function is very expensive; hence it is another point to be refined in later versions.

In [ ]:
Class GameState(GameState):    
    def getValidMoves(self): 
        moves = self.getAllPossibleMoves()
        for i in range(len(moves)-1,-1,-1):
            self.makeMove(moves[i])
            self.whiteToMove = not self.whiteToMove
            if self.inCheck():
                moves.remove(moves[i])
                #very inefficient, popping a non-end-list element!
            self.whiteToMove = not self.whiteToMove
            self.undoMove()
        if len(moves)==0:   #checkmate or stalemate
            if self.inCheck():
                self.checkmate=True
            else:
                self.stalemate=True
        else:
            self.checkmate=False
            self.stalemate=False
        return moves